## Overview

This workbook contains code and notes used to answer week 3's programming assignment.

In [1]:
# Need to load the genome
from Bio.Seq import Seq
import Bio.SeqIO

genome = list(Bio.SeqIO.parse('chr1.GRCh38.excerpt.fasta', 'fasta')).pop().seq

In [50]:
# Prefer to use numpy than a list of lists
import numpy as np

# We also need an approximate matching algorithm
# As mentioned in the programming notes, this is similar to the exact alignment algorithm
# but critically, the first row is initialized as zeros.
def approximate_match(P, T):
    """
    Adaptation of the edit distance function to do approximate matching
    I renamed the variables to be more telling than X/Y.
    Also, I leveraged numpy arrays instead of lists of lists,
    although there is little difference in efficiency in this case
    
    Args:
        P (str): pattern to match to T
        T (str): reference sequence (e.g., genome)
    
    Returns:
        edit_distance (integer): approximate match distance
    """

    # Need len + 1 so we can account for the initialization term
    D = np.zeros([len(P)+1, len(T)+1]).astype(int)

    D[:, 0] = range(len(P)+1)

    for row in range(1, len(P)+1):
        for col in range(1, len(T)+1):
            dist_vert = D[row-1, col] + 1
            dist_hor = D[row, col-1] + 1

            # Compare against last alignment
            dist_diag = D[row-1, col-1]

            # Do the letters mismatch?
            if P[row-1] == T[col-1]:
                dist_diag += 0
            else:
                dist_diag += 1

            # Finally, assign the distance to this particular cell in the array
            D[row, col] = min(dist_vert, dist_hor, dist_diag)

    return D[-1, :].min()

In [68]:
list(Bio.SeqIO.parse('chr1.GRCh38.excerpt.fasta', 'fasta'))

'CM000663.2_excerpt'

## Example 01

Example provided in the programming reading section. Testing function above against it to make sure it works as expected

In [52]:
P = 'GCGTATGC'
T = 'TATTGGCTATACGGTT'

approximate_match(P, T)

2

## Question 01

In [58]:
P = 'GCTGATCGATCGTACG'
approximate_match(P, genome)

3

## Question 02

In [60]:
P = 'GATTTACCAGATTGAG'
approximate_match(P, genome)

2

## Question 03

In [118]:
# This contains all the reads in the file
genome = list(Bio.SeqIO.parse('ERR266411_1.for_asm.fastq', 'fastq')) # subset for testing purposes

In [94]:
genome[0]

SeqRecord(seq=Seq('TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGA...CTC', SingleLetterAlphabet()), id='ERR266411.1', name='ERR266411.1', description='ERR266411.1 HS18_09233:8:1307:10911:3848#168/1', dbxrefs=[])

In [61]:
def overlap(seq_suffix, seq_prefix, min_length=3):
    """
    Check to see if the suffix of seq_suffix matches the prefix
    of seq_prefix exactly. Note that no differences are tolerated.
    
    Args:
        seq_suffix (str): sequence whose suffix will be compared
                          against the seq_prefix
        seq_prefix (str): sequence whose prefix will be compared
                          against the seq_suffix

    Returns:
        is_overlap (bool): True if the sequences match exactly
                           up to min_length. Otherwise, False
    """

    start = 0  # start all the way at the left

    while True:
        # 
        start = seq_suffix.find(seq_prefix[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return False
        # found occurrence; check for full suffix/prefix match
        if seq_prefix.startswith(seq_suffix[start:]):
            return True

        start += 1

In [139]:
# As mentioned in question 3, it is very slow to compare all possible combinations of reads from even a small
# set of reads. Instead, build an index out of the reads. To shorten our search path, we'll build a custom
# index class that will create a lookup for the desired k-mer.
#
# To do this, I modified previous work using a k-mer index.
class ReadDict(object):
    """
    Wanted a lighter-weight way of storing relationships between
    IDs and sequences, so decided to create a class.
    
    Reads are stored as a dict internally.
    """

    def __init__(self, reads):
            
        self.reads = reads
        
        self.read_dict = {}
        
        for seq_record in reads:
            
            self.read_dict[seq_record.id] = seq_record.seq

    def get_read(self, read_id):
        
        return self.read_dict[read_id]


class Index(object):
    
    # Build a k-mer index
    def __init__(self, reads, k):

        # Track k_mer length
        self.k = k
        
        # For every read, build the index
        self.index = self._get_read_kmers(reads, k)

    def _get_read_kmers(self, reads, k):

        k_mers = {}

        # Loop through all the reads
        for seq_record in reads:
 
            seq = seq_record.seq
            seq_id = seq_record.id

            for i in range(len(seq) - k + 1):  # for each k-mer

                key = str(seq[i:i+k])

                if key not in k_mers:
                    k_mers[key] = set()

                k_mers[key].add(seq_id)

        return k_mers


# Now, use an index and a ReadDict to do the comparisons
def overlap_fast(seq_record, index, read_dict, min_length=3):
    """
    Check to see if the suffix of seq_suffix matches the prefix
    of seq_prefix exactly. Note that no differences are tolerated.
    
    Args:
        seq_suffix (str): sequence whose suffix will be compared
                          against the seq_prefix
        seq_prefix (str): sequence whose prefix will be compared
                          against the seq_suffix

    Returns:
        is_overlap (bool): True if the sequences match exactly
                           up to min_length. Otherwise, False
    """

    # Get the suffix length
    suffix = seq_record.seq[-1*min_length:]
    
    # Get a list of sequences to search
    #  This will return a list of sequence IDs to compare against
    # Note: we need to remove self-referential comparisons
    # So, remove the desired seq_record from the set
    # This is done using a set difference
    seq_subset = index.index[suffix] - set(seq_record.id)
    
    overlap_seq = set()

    for seq_id in seq_subset:
        
        if overlap(seq_record.seq, read_dict.get_read(seq_id), min_length):
            
            overlap_seq.add(seq_id)

    return overlap_seq

In [147]:
# Build the index
min_length = 30

# This is costly to build initially
index = Index(genome, min_length)
read_dict = ReadDict(genome)

In [150]:
# Now, find matching sequences for a test case
seq_record = genome[0]
overlap_seq = overlap_fast(seq_record, index, read_dict, min_length)

/Users/cbishop/.virtualenvs/python3new/lib/python3.6/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [154]:
overlap_seq

{'ERR266411.1', 'ERR266411.15', 'ERR266411.4'}

In [141]:
min_length = 3
seq_record = genome[0]
seq_record.seq[-1*min_length:]
seq_record.seq
overlap_seq = overlap_fast(seq_record, index, read_dict)

/Users/cbishop/.virtualenvs/python3new/lib/python3.6/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [144]:
overlap(seq_record.seq, read_dict.get_read('ERR266411.1'), 3)

True

In [143]:
seq_record

SeqRecord(seq=Seq('TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGA...CTC', SingleLetterAlphabet()), id='ERR266411.1', name='ERR266411.1', description='ERR266411.1 HS18_09233:8:1307:10911:3848#168/1', dbxrefs=[])

In [142]:
overlap_seq

{'ERR266411.1',
 'ERR266411.1048',
 'ERR266411.1066',
 'ERR266411.12645',
 'ERR266411.12659',
 'ERR266411.12740',
 'ERR266411.12742',
 'ERR266411.12762',
 'ERR266411.12764',
 'ERR266411.12812',
 'ERR266411.12828',
 'ERR266411.12829',
 'ERR266411.12834',
 'ERR266411.13180',
 'ERR266411.1441',
 'ERR266411.15',
 'ERR266411.15724',
 'ERR266411.15726',
 'ERR266411.15730',
 'ERR266411.15733',
 'ERR266411.15743',
 'ERR266411.15769',
 'ERR266411.15794',
 'ERR266411.15874',
 'ERR266411.15985',
 'ERR266411.17428',
 'ERR266411.19323',
 'ERR266411.19484',
 'ERR266411.19560',
 'ERR266411.19589',
 'ERR266411.19610',
 'ERR266411.19645',
 'ERR266411.19890',
 'ERR266411.20001',
 'ERR266411.21605',
 'ERR266411.22069',
 'ERR266411.22112',
 'ERR266411.22159',
 'ERR266411.22164',
 'ERR266411.22166',
 'ERR266411.22207',
 'ERR266411.22209',
 'ERR266411.22296',
 'ERR266411.22300',
 'ERR266411.22349',
 'ERR266411.22353',
 'ERR266411.22532',
 'ERR266411.22658',
 'ERR266411.2301',
 'ERR266411.27923',
 'ERR266411

In [132]:
index.index['CTC'] - {'ERR266411.56615'}

{'ERR266411.16592',
 'ERR266411.59599',
 'ERR266411.60531',
 'ERR266411.19038',
 'ERR266411.28399',
 'ERR266411.211',
 'ERR266411.8199',
 'ERR266411.6118',
 'ERR266411.3314',
 'ERR266411.64616',
 'ERR266411.52355',
 'ERR266411.73460',
 'ERR266411.63867',
 'ERR266411.25176',
 'ERR266411.62639',
 'ERR266411.33531',
 'ERR266411.13231',
 'ERR266411.14611',
 'ERR266411.10165',
 'ERR266411.48253',
 'ERR266411.28527',
 'ERR266411.5801',
 'ERR266411.49120',
 'ERR266411.74784',
 'ERR266411.43998',
 'ERR266411.73894',
 'ERR266411.61808',
 'ERR266411.69952',
 'ERR266411.62562',
 'ERR266411.65836',
 'ERR266411.61652',
 'ERR266411.26375',
 'ERR266411.31986',
 'ERR266411.39816',
 'ERR266411.72579',
 'ERR266411.62096',
 'ERR266411.10047',
 'ERR266411.39917',
 'ERR266411.54358',
 'ERR266411.39783',
 'ERR266411.58308',
 'ERR266411.29371',
 'ERR266411.65601',
 'ERR266411.69063',
 'ERR266411.165',
 'ERR266411.20500',
 'ERR266411.17409',
 'ERR266411.35258',
 'ERR266411.49553',
 'ERR266411.72898',
 'ERR266

In [119]:
index = Index(genome, k=3)
index._get_read_kmers(genome, 3)
# x = index._get_read_kmers(genome[0], 3)
# y = index._get_read_kmers(genome[1], 3)

{'TAA': {'ERR266411.56615',
  'ERR266411.16592',
  'ERR266411.59599',
  'ERR266411.60531',
  'ERR266411.19038',
  'ERR266411.28399',
  'ERR266411.9195',
  'ERR266411.211',
  'ERR266411.8199',
  'ERR266411.6118',
  'ERR266411.37035',
  'ERR266411.3314',
  'ERR266411.64616',
  'ERR266411.11705',
  'ERR266411.52355',
  'ERR266411.73460',
  'ERR266411.63867',
  'ERR266411.25176',
  'ERR266411.16092',
  'ERR266411.62639',
  'ERR266411.33531',
  'ERR266411.8098',
  'ERR266411.13231',
  'ERR266411.14611',
  'ERR266411.10165',
  'ERR266411.41969',
  'ERR266411.48253',
  'ERR266411.28527',
  'ERR266411.5801',
  'ERR266411.49120',
  'ERR266411.74784',
  'ERR266411.45227',
  'ERR266411.43998',
  'ERR266411.61808',
  'ERR266411.15842',
  'ERR266411.69952',
  'ERR266411.62562',
  'ERR266411.65836',
  'ERR266411.61652',
  'ERR266411.26375',
  'ERR266411.31986',
  'ERR266411.39816',
  'ERR266411.72579',
  'ERR266411.62096',
  'ERR266411.10047',
  'ERR266411.39917',
  'ERR266411.54358',
  'ERR266411.1

In [111]:
y

{'AAC': {'ERR266411.2'},
 'ACA': {'ERR266411.2'},
 'CAA': {'ERR266411.2'},
 'AAG': {'ERR266411.2'},
 'AGC': {'ERR266411.2'},
 'GCA': {'ERR266411.2'},
 'CAG': {'ERR266411.2'},
 'AGT': {'ERR266411.2'},
 'GTA': {'ERR266411.2'},
 'TAG': {'ERR266411.2'},
 'TAA': {'ERR266411.2'},
 'AAT': {'ERR266411.2'},
 'ATT': {'ERR266411.2'},
 'TTC': {'ERR266411.2'},
 'TCC': {'ERR266411.2'},
 'CCT': {'ERR266411.2'},
 'CTG': {'ERR266411.2'},
 'TGC': {'ERR266411.2'},
 'GCT': {'ERR266411.2'},
 'CTT': {'ERR266411.2'},
 'TTT': {'ERR266411.2'},
 'TTA': {'ERR266411.2'},
 'TAT': {'ERR266411.2'},
 'ATC': {'ERR266411.2'},
 'TCA': {'ERR266411.2'},
 'AGA': {'ERR266411.2'},
 'GAT': {'ERR266411.2'},
 'ATA': {'ERR266411.2'},
 'TCG': {'ERR266411.2'},
 'CGA': {'ERR266411.2'},
 'GAC': {'ERR266411.2'},
 'ACT': {'ERR266411.2'},
 'CTC': {'ERR266411.2'},
 'CAT': {'ERR266411.2'},
 'GAA': {'ERR266411.2'},
 'AAA': {'ERR266411.2'},
 'TAC': {'ERR266411.2'},
 'ACG': {'ERR266411.2'},
 'GTG': {'ERR266411.2'},
 'TGT': {'ERR266411.2'},
